In [1]:
import pandas as pd
import numpy as np
data = pd.DataFrame({'a': [1, 2, 3, 4, 5],'b': [1, -2, 5.5, -7.9, 3],'c': [-2, 0., 3, -8.3, -2.]})
data

,a,b,c
0,1,1.0,-2.0
1,2,-2.0,0.0
2,3,5.5,3.0
3,4,-7.9,-8.3
4,5,3.0,-2.0


In [2]:
data.columns

Index(['a', 'b', 'c'], dtype='object')

In [3]:
data.values

array([[ 1. ,  1. , -2. ],
       [ 2. , -2. ,  0. ],
       [ 3. ,  5.5,  3. ],
       [ 4. , -7.9, -8.3],
       [ 5. ,  3. , -2. ]])

In [4]:
df2 = pd.DataFrame(data.values, columns=['z', 'y', 'x'])
df2

,z,y,x
0,1.0,1.0,-2.0
1,2.0,-2.0,0.0
2,3.0,5.5,3.0
3,4.0,-7.9,-8.3
4,5.0,3.0,-2.0


In [5]:
df2['strings'] = ['fewfe', 'frfre', 'drrwgw', 'rgt3r', 'g53h']

In [6]:
df2

,z,y,x,strings
0,1.0,1.0,-2.0,fewfe
1,2.0,-2.0,0.0,frfre
2,3.0,5.5,3.0,drrwgw
3,4.0,-7.9,-8.3,rgt3r
4,5.0,3.0,-2.0,g53h


In [7]:
df2.values

array([[1.0, 1.0, -2.0, 'fewfe'],
       [2.0, -2.0, 0.0, 'frfre'],
       [3.0, 5.5, 3.0, 'drrwgw'],
       [4.0, -7.9, -8.3, 'rgt3r'],
       [5.0, 3.0, -2.0, 'g53h']], dtype=object)

In [9]:
data

,a,b,c
0,1,1.0,-2.0
1,2,-2.0,0.0
2,3,5.5,3.0
3,4,-7.9,-8.3
4,5,3.0,-2.0


In [10]:
import patsy
c, X = patsy.dmatrices('c ~ a + b', data)
c

DesignMatrix with shape (5, 1)
     c
  -2.0
   0.0
   3.0
  -8.3
  -2.0
  Terms:
    'c' (column 0)

In [11]:
np.asarray(c)

array([[-2. ],
       [ 0. ],
       [ 3. ],
       [-8.3],
       [-2. ]])

In [12]:
np.asarray(X)

array([[ 1. ,  1. ,  1. ],
       [ 1. ,  2. , -2. ],
       [ 1. ,  3. ,  5.5],
       [ 1. ,  4. , -7.9],
       [ 1. ,  5. ,  3. ]])

In [14]:
coef, resid, _, _ = np.linalg.lstsq(X, c)

E:\ana\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [15]:
coef

array([[ 0.29916343],
       [-0.70171643],
       [ 0.67517668]])

In [16]:
c, X = patsy.dmatrices('c ~ standardize(a) + center(b)', data)

In [17]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(a)  center(b)
          1        -1.41421       1.08
          1        -0.70711      -1.92
          1         0.00000       5.58
          1         0.70711      -7.82
          1         1.41421       3.08
  Terms:
    'Intercept' (column 0)
    'standardize(a)' (column 1)
    'center(b)' (column 2)

In [18]:
X = patsy.build_design_matrices([X.design_info], data)
X

[DesignMatrix with shape (5, 3)
   Intercept  standardize(a)  center(b)
           1        -1.41421       1.08
           1        -0.70711      -1.92
           1         0.00000       5.58
           1         0.70711      -7.82
           1         1.41421       3.08
   Terms:
     'Intercept' (column 0)
     'standardize(a)' (column 1)
     'center(b)' (column 2)]

In [19]:
c, X = patsy.dmatrices('c ~ I(a + b)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(a + b)
          1       2.0
          1       0.0
          1       8.5
          1      -3.9
          1       8.0
  Terms:
    'Intercept' (column 0)
    'I(a + b)' (column 1)

In [20]:
data['b'] = data['b'].map({0: 'zero', 1: 'one'})

In [21]:
data

,a,b,c
0,1,one,-2.0
1,2,NaN,0.0
2,3,NaN,3.0
3,4,NaN,-8.3
4,5,NaN,-2.0


In [22]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
        return mean + np.sqrt(variance) * np.random.randn(*size)
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
 dnorm(0, 0.6, size=N),
 dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]
y = np.dot(X, beta) + eps
X[:5]


array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [23]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Sat, 16 Mar 2019   Prob (F-statistic):           7.44e-12
Time:                        20:00:21   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

In [25]:
data = pd.DataFrame(X, columns=['xx', 'yy', 'zz'])
data['y'] = y
data[:5]

,xx,yy,zz,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941
